# Week 09 (?!) LA Neighborhood Transit: Spatial Statistics Overview

**By:** Andrew Williams and Ben Brassette

Uploaded without runnnig the cells as I'm having difficulty saving my notebooks after I run them, even if I'm using minimal memory. Will contact Ben Winjum about this.

**Decription:** Purpose of this notebook is to use the tools from the Week 08 Spatial Stats lesson in order to provide a better analysis of our chosen neighborhoods for this project. 

**Neighborhoods:**
* Downtown (Central LA)
* Pico-Union(Central LA)
* Panaroma City (San Fernando Valley)
* North Hollywood (San Fernando Valley)
* Mid-City (Central LA, Car Dominant)

# Library 

In [ ]:
# to read and wrangle data
import pandas as pd

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

# Trimming Data

## Data Check

I'm going to downloand my dataset that features mode of transportatotion to work. I'm using a dataset that also has neighborhoods, income, and racial breakdownn in case I need to explore other variables (time permitting). I will do a typical check of the data to make sure it's ready for some exploration. 

In [ ]:
gdf= gpd.read_file('m2w_income_race_new.geojson')

In [ ]:
type(gdf)

In [ ]:
gdf.shape

In [ ]:
gdf. head(4)

In [ ]:
gdf.tail(4)

I'll need to rename my columns

In [ ]:
gdf.columns.to_list()

In [ ]:
gdf.columns=['Geoid',
 'Name',
 'Neighborhood',
 'Median Inc',
 'Total Work',
 'Car Total',
 'Drove alone',
 'Carpooled',
 'Public transportation',
 'Bus',
 'Subway',
 'Long-distance rail',
 'Light rail',
 'Worked from home',
 '%Car Total',
 '%Drove alone',
 '%Carpooled',
 '%Public transportation',
 '%Bus',
 '%Subway',
 '%Long-distance rail',
 '%Light rail',
 '%Worked from home',
 'Total Pop',
 'White',
 'Black',
 'Native',
 'Asian',
 'Native H',
 'Hispanic or Latino',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Hawaiian',
 '%Hispanic or Latino',
 'geometry']

In [ ]:
gdf.head(3)

All is right with the world and the dataset is good to go!

# Normalizing: Our Data per 1000 people

Following the example from class, I'm normalizing a couple variables to see the rate per 1000 people

In [ ]:
gdf['car_per_1000'] = gdf['Car Total']/gdf['Total Work']*1000
gdf['transit_per_1000'] = gdf['Public transportation']/gdf['Total Work']*1000
gdf['bus_per_1000'] = gdf['Bus']/gdf['Total Work']*1000
gdf['subway_per_1000'] = gdf['Subway']/gdf['Total Work']*1000
gdf['disrail_per_1000'] = gdf['Long-distance rail']/gdf['Total Work']*1000
gdf['lightrail_per_1000'] = gdf['Worked from home']/gdf['Total Work']*1000

Well we use all of these, no. Likely just car and transporation. But it's nice to have options.

In [ ]:
gdf.sample(3)

Also note, I should really stops adding space to my variables. 

In [ ]:
gdf.sort_values(by="transit_per_1000").tail(10)

So I'm following Yoh's notebook until I get my feet settled with this data, but did not realize 5 tracts have no data. They are not in our slected neighborhoods, but would like to explore why these show up with no values. I know one of the tracts consists of the beach on the Westside. I imagine other are similar in nature.

I know people are using points for their data with their polygons but I'm going to continue to use my polygon tracts. HOWEVER, it would be interesting to to map bus stops or transit stops in each tract. I initially tried bus stops but was having troubles uploading my data to jupyter. May try again later. I could use rail stops, but given the dismal rail ridership I've seen, I'm not sure if that will be terribly helpful. Will forge on for better or worse now

In [ ]:
fig,ax = plt.subplots(figsize=(20,18))
gdf.sort_values(by='transit_per_1000',ascending=False)[:30].plot(ax=ax,
                                                                 color='blue',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)


# title
ax.set_title('Top 30 Tracts of Transit Ridership per 1000 people')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax, crs=gdf.crs.to_string())

I changed the contexuality input that we were using from class as that wasn't showing anything. I think there may be an issue with projecting my data to a CRS, but am not entirely sure.

Top 30 tracts are in the Central LA. It looks like mostly Westlake with some scattering around the edges. It's noticable that these areas are presenting themselves as clusters, with one mega-cluster in Westlake.

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

gdf.plot(ax=ax,
        column='transit_per_1000',
        legend=True,
        alpha=0.8,
        cmap='cividis',
        scheme='quantiles')

ax.axis('off')
ax.set_title('Transit Ridership Per 1000 People',fontsize=22) #font size! Well hot dog. Going to be using this for the next week
ctx.add_basemap(ax, crs=gdf.crs.to_string())

Okay, really want to get my bus stop data to work now, I think that would be helpful. Still having trouble with the data itself.

But the story: High transit use in Central LA and South LA and moderate usage on the Westside and the San Fernando Valley. This presents a new persective my adding normalizing the data per 1000 people, which in effect is a different way to present percentages. Still interesting to see. I'm curious what the lag data will show. 

# Global Spaitial Autocorrelation or Something Like That

So I'm using K to count the number of nearest neighbors. When we eventually get down to some of the mapping and charts, I think seeing clusters of transit will provide some insights, but am stil worried about using just the one combined dataset I have-- as it feels "flat."

In [ ]:
# calculate spatial weight
wq =  lps.weights.KNN.from_dataframe(gdf,k=8)

# Row-standardization
wq.transform = 'r'

Woo! Something happened. 

Doing stuff with spatial lag. Kind of exciting to see what happens with this. 

Moved down to just the one variable to make sure I get this right. But creating a new variable

In [ ]:
gdf['transit_per_1000_lag'] = lps.weights.lag_spatial(wq, gdf['transit_per_1000'])

In [ ]:
gdf.sample(10)[['Total Work','Neighborhood','Public transportation','transit_per_1000','transit_per_1000_lag']]

Oh! This is what I was expecting and I'm also surprised. There will be a couple layers to unpack here in a bit. Excited to move on.

## DONUT and DONUT HOLE TIME (down with diamonds!)

Going to try and identify some donuts and donut holes. 

In [ ]:
gdf['transit_lag_diff'] = gdf['transit_per_1000'] - gdf['transit_per_1000_lag']

In [ ]:
gdf.sort_values(by='transit_lag_diff')

Well that query wasn't too helpful and I'm definately not going to check out the whole dataset. Though myabe it's time to check out what this means for our selected neighborhoods.

* Downtown (Central LA)
* Pico-Union(Central LA)
* Panaroma City (San Fernando Valley)
* North Hollywood (San Fernando Valley)
* Mid-City (Central LA, Car Dominant)

In [ ]:
gdf.query("Neighborhood== 'Downtown'").sort_values(by='transit_lag_diff')

Obvisouly these are all in one neighborhood, but I will be interested to see how they spatailly related to each other. There is a pretty significant range in transit lag differnces.

In [ ]:
gdf.query("Neighborhood== 'Pico-Union'").sort_values(by='transit_lag_diff')

Range is not quite as large as Downtown, but still significant.

In [ ]:
gdf.query("Neighborhood== 'Panorama City'").sort_values(by='transit_lag_diff')

Panorama City range is actually similar to Pico-Union, which I do find suprising. Excited to plot these soon and see their spatial relation>

In [ ]:
gdf.query("Neighborhood== 'North Hollywood'").sort_values(by='transit_lag_diff')

Less transit lag differences that are positive, but can be expected given this neighborhood is in the San Fernando Valley.

In [ ]:
gdf.query("Neighborhood== 'Mid-City'").sort_values(by='transit_lag_diff')

Again, seems similar to Pico-Union and Panorama City. Maybe these patterns are indicative of neighborhoods in general, or at least neighborhoods with marginally more transit ridership.

In [ ]:
gdf_donut = gdf.sort_values(by='transit_lag_diff').head(5)
gdf_donut

In [ ]:
# hashtag-donut holes for the win
gdf_donuthole = gdf.sort_values(by='transit_lag_diff').tail(28)
gdf_donuthole

So the last 28 tracts have NaN values. I wonder if that's becasue trasnit ridership in these areas is so small. I thought they would show up as negative values, so I'm a little confused in what's happening here. 

<div class="alert alert-danger">
<h1>Note</h1>
The maps below all have the wrong column value `transit_lag_diff` so changed those to `transit_per_1000_lag`
</div>

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
gdf.plot(ax=ax[0], # this assigns the map to the left subplot
         column='transit_per_1000', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.8, 
         legend=True,)


ax[0].axis("off")
ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
gdf.plot(ax=ax[1],
         column='transit_per_1000_lag',
         scheme='quantiles',
         k=5, 
         edgecolor='white',
         linewidth=0, 
         alpha=0.8,
         legend=True,)


ax[1].axis("off")
ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

plt.show()

So, I defintaly spend too much time exploring different color options. It's JUST SO FUN. 

I'm also not delving into donut and donut holes as I would like, but need to be a bit more efficient with my time (or at least try to).

There is a much more clear picture now. Strong clusterings in Central LA and the SF Valley. Moderate use in West LA and even some in west SF Valley--all these being relative given low trasnit numbers in general.

Mapping the Neighborhoods
* Downtown (Central LA)
* Pico-Union(Central LA)
* Panoroma City (San Fernando Valley)
* North Hollywood (San Fernando Valley)
* Mid-City (Central LA, Car Dominant)

In order to save on memory and successfully save this file,the below maps are disabled--I added a # to the first run line of code

Downtown

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
gdf.query("Neighborhood== 'Downtown'").plot(ax=ax[0], # this assigns the map to the left subplot
         column='transit_per_1000', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.8, 
         legend=True,)


ax[0].axis("off")
ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
gdf.query("Neighborhood== 'Downtown'").plot(ax=ax[1],
         column='transit_per_1000_lag',
         scheme='quantiles',
         k=5, 
         edgecolor='white',
         linewidth=0, 
         alpha=0.8,
         legend=True,)


ax[1].axis("off")
ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

plt.show()

* Numbers significantly reduced
* Other tracts not on this neighborhood are likely influencing this the boundaries of each neighborhood.
* Auto travel of neighboring tracts are likely influencing these numbers
* Ultimately, surprising to see how trasnit lag numbers on this "high transit" area, though since its' LA, maybe not so surprising. I also think looking on a finer detail would help--block groups


Pico-Union

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
gdf.query("Neighborhood== 'Pico-Union'").plot(ax=ax[0], # this assigns the map to the left subplot
         column='transit_per_1000', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.8, 
         legend=True,)


ax[0].axis("off")
ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
gdf.query("Neighborhood== 'Pico-Union'").plot(ax=ax[1],
         column='transit_per_1000_lag',
         scheme='quantiles',
         k=5, 
         edgecolor='white',
         linewidth=0, 
         alpha=0.8,
         legend=True,)


ax[1].axis("off")
ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

plt.show()

So be warned, I'm imaging all of these maps are likely going to say the same thing, more or less. These other neighborhoods will have significantly less transit compared to Downtown.
* It would be helpful to see what neighborhoods surround Pico-Union.
* Auto travel of neighboring tracts are likely influencing these numbers
* Most of these look fairly uniform


Panorama City

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
gdf.query("Neighborhood== 'Panorama City'").plot(ax=ax[0], # this assigns the map to the left subplot
         column='transit_per_1000', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.8, 
         legend=True,)


ax[0].axis("off")
ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
gdf.query("Neighborhood== 'Panorama City'").plot(ax=ax[1],
         column='transit_per_1000_lag',
         scheme='quantiles',
         k=5, 
         edgecolor='white',
         linewidth=0, 
         alpha=0.8,
         legend=True,)


ax[1].axis("off")
ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

plt.show()

* Numbers reduced, more tracts are in the "positive", I should look at their total populations. This is surprising given this neighborhood’s location in the San Fernando Valley
* It would be helpful to see what neighborhoods surround Panorama City.
* Auto travel of neighboring tracts are likely influencing these numbers


North Hollywod

In [ ]:
# create the 1x2 subplots
#fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
#gdf.query("Neighborhood== 'North Hollywood'").plot(ax=ax[0], # this assigns the map to the left subplot
       #  column='transit_per_1000', 
      #   scheme='quantiles',
     #    k=5, 
    #     edgecolor='white', 
   #      linewidth=0, 
  #       alpha=0.8, 
 #        legend=True,)


#ax[0].axis("off")
#ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
#gdf.query("Neighborhood== 'North Hollywood'").plot(ax=ax[1],
        # column='transit_per_1000_lag',
       #  scheme='quantiles',
      #   k=5, 
     #    edgecolor='white',
    #     linewidth=0, 
  ##       alpha=0.8,
 #        legend=True,)


#ax[1].axis("off")
#ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

#plt.show()

So these maps are practically identical, with some slight changes
* Again, more tracts in the positive
* That is unique from what I've seen here. There are some nuances to this, but something I was not expecting.
* Interval ranges are also much more condensed compared to the others

Mid-City (Not running map in order to save room to save)

In [ ]:
# create the 1x2 subplots
#fig, ax = plt.subplots(1, 2, figsize=(20, 15))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
#gdf.query("Neighborhood== 'Mid-City'").plot(ax=ax[0], # this assigns the map to the left subplot
       #  column='transit_per_1000', 
      #   scheme='quantiles',
      #   k=5, 
       #  edgecolor='white', 
       #  linewidth=0, 
        # alpha=0.8, 
         #legend=True,)
#

#ax[0].axis("off")
#ax[0].set_title("Transit per 1000",fontsize=22)

# spatial lag map on the right
#gdf.query("Neighborhood== 'Mid-City'").plot(ax=ax[1],
        # column='transit_per_1000_lag',
       #  scheme='quantiles',
       #  k=5, 
        # edgecolor='white',
        # linewidth=0, 
        # alpha=0.8,
        # legend=True,)


#ax[1].axis("off")
#ax[1].set_title('Transit Spatial Lag, Per 1000 People',fontsize=22)

#plt.show()

* Again, almost identical.
* Numbers are compressed. Downtown appears to have the biggest range of trasnit numbers.

# Moran

This part I'm again not condifent in. Will follow Yoh's notebook for guidance and see what we can find.

Restarting some steps removing NaN values from the dataset. I'm dropping these values, which I realize is a drastic option. I'm also trying some options like a trimmed dataset or just excluding them.

In [ ]:
gdf2=gdf

In [ ]:
gdf2.sample(5)

In [ ]:
gdf2=gdf.drop([1003, 1001, 998, 997, 995])

In [ ]:
gdf2.sort_values(by='transit_per_1000').head()

In [ ]:
gdf2.sort_values(by='transit_per_1000').tail()

In [ ]:
gdf2.reset_index()

In [ ]:
gdf2=gdf2.reset_index()

In [ ]:
gdf2.sample()

So we dropped the NaN neighborhoods. I believe that was the crux of our problem, but I'm not entirely sure. 

Significant problems right here with getting Moran to work. If I'm unable to solve this and the TAs/Yoh are unavaiable, I will submit the notebook as is, depite the lack of moran related charts and maps we were supposed to use. I will circle back around with them later this week to try and get some help. Apologies for coming up short. 

<div class="alert alert-danger">
<h1>Note</h1>
You are misssing some steps here...
    
First do the global autocorrelation
</div>

In [ ]:
# calculate spatial weight
wq =  lps.weights.KNN.from_dataframe(gdf2,k=8)

# Row-standardization
wq.transform = 'r'

In [ ]:
y = gdf2.transit_per_1000
moran = Moran(y, wq)
moran.I

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.show()

There is a very clear patter pattern here. Though there is huge concentration at 0,0, which I think explains general low public transportation use in LA. Stong clope. 

In [ ]:
plot_moran_simulation(moran,aspect_equal=False)

In [ ]:
moran.p_sim

Low p-value. Would reject my null hpothesis based on this with a chosen alpha of 0.05. Public trasnportation is not equal among census tracts.

<div class="alert alert-danger">
<h1>Note</h1>
Now you can do the local auto-correlation
</div>

In [ ]:
# calculate local moran values
lisa = esda.moran.Moran_Local(y, wq)

In [ ]:
# Plot
fig,ax = plt.subplots(figsize=(10,15))

moran_scatterplot(lisa, ax=ax, p=0.05)
ax.set_xlabel("Arrests")
ax.set_ylabel('Spatial Lag of Public Transportation')

# add some labels
plt.text(1.95, 0.5, "HH", fontsize=25)
plt.text(1.95, -1, "HL", fontsize=25)
plt.text(-2, 1, "LH", fontsize=25)
plt.text(-1, -1, "LL", fontsize=25)
plt.show()

Shows P-values that are less than 0.05--these are statisically significant, spatailly autocorrelated geographies.My assumption is that these will all be in Central LA, few a few scattered elsewhere--SF Valley. I assum the LL points will be in the SF Valley as well and in the harbor region.

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))
lisa_cluster(lisa, gdf2, p=0.05, ax=ax)
plt.show()

This shows what I assumed would happen, but goes further. I am particularly interested in the HL and LH clusters. What explains the HL on the norwest of the SF Valley? The HH points have statisically confirmed what we know, but narrowed the area in which we thhought are high transit users. 

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

# regular count map on the left
lisa_cluster(lisa, gdf2, p=0.05, ax=ax[0])

ax[0].axis("off")
ax[0].set_title("P-value: 0.05")

# spatial lag map on the right
lisa_cluster(lisa, gdf2, p=0.01, ax=ax[1])
ax[1].axis("off")
ax[1].set_title("P-value: 0.01")

plt.show()

If we change our p-value to 0.01, are results are narrowed even further. HH transit areas are only in Central LA, which itself has eroded extensively. In all honesty, I'm surprised that any SF Valley tracts showed up with a p-value of 0.05, but makes me feel better knowing we have a focus in that region in our analysis. I think this adds a new element to what we've done thus far by showing the stasitical signifance of high transit tracts. 

# Division of Labor

Both discussed the process and communicated analysis of data. 

**Andrew:** Prepped the data, provided an added lens to the neighborhoods we're focusing on, relayed info to Ben, AND struggled with Moran.  

**Ben:** Using the same data, also ran tests to explore data and practice skills, provided mores analysis, and also struggled with Moran.  

**Yoh:** Fixed maps and added global autocorrelation to the new dataset so we could run Moran.